# LeNet5 A-Z mnist

##### Preprocess

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
data = pd.read_csv(r'C:\Users\User\Desktop\Learn-Machine-Learning\Intro_to_DLNN\DL_A-Zmnist -VGG16\Dataset\handwritten_data_785.csv')
data = data.to_numpy()


Random 10% of data

In [ ]:
cut = np.random.permutation(np.arange(len(data)))
split0 = data.shape[0] * 7//10
idx = cut[:split0]
data_suff = data[idx]
del data
print(len(data_suff))

In [ ]:
import string
X = data_suff[:,1:]
Y = data_suff[:,0]
nr_to_letter = {k:v.upper() for k,v in enumerate(list(string.ascii_lowercase))}
X=X.reshape(len(X),28,28)
X = X[:,:,:,None]
print(X.shape)
print(Y)
print(nr_to_letter)

In [ ]:
n = 10
for j,i in enumerate(np.random.choice(len(X),n)):
    plt.subplot(1,n,j+1)
    plt.imshow(X[i],cmap='gray')
    plt.axis('off')
plt.show()

In [ ]:
X = tf.image.resize(X,[32,32])

Split to 80% train and 20% test.

In [ ]:
idx = np.random.permutation(np.arange(len(X)))
split = X.shape[0] * 8//10 # แบ่ง train 80 %
train_idx = idx[:split]
test_idx = idx[split:]
Xtrain,Ytrain,Xtest,Ytest = [],[],[],[]
for idx in train_idx:
    Xtrain.append(X[idx]/255.)
    Ytrain.append(Y[idx].astype(np.int))
Xtrain = np.array(Xtrain)    
for idx in test_idx:
    Xtest.append(X[idx]/255.)
    Ytest.append(Y[idx].astype(np.int))  
Xtrain,Ytrain,Xtest,Ytest = np.array(Xtrain) ,np.array(Ytrain) ,np.array(Xtest) ,np.array(Ytest)      
# Xtrain,Ytrain = X[train_idx]/255.,Y[train_idx].astype(np.int)
# Xtest,Ytest = X[test_idx]/255.,Y[test_idx].astype(np.int)
print(Xtrain.shape)
print(Xtest.shape)

In [ ]:
n = 10
for j,i in enumerate(np.random.choice(len(Xtrain),n)):
    plt.subplot(1,n,j+1)
    plt.imshow(Xtrain[i],cmap='gray')
    plt.axis('off')
plt.show()

##### Training

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),padding="same",activation="relu",input_shape=(224,224,3)))
model.add(tf.keras.layers.Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(256,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(256,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(256,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(512,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(512,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3,3),padding="same",activation="relu"))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4096,activation="relu"))
model.add(tf.keras.layers.Dense(4096,activation="relu"))
model.add(tf.keras.layers.Dense(10,activation="softmax"))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.sparse_categorical_crossentropy)

In [ ]:
(Xtrain, Ytrain), (Xtest, Ytest) = tf.keras.datasets.mnist.load_data()
Xtrain = Xtrain[:, :, :, None]/255.
Xtest = Xtest[:, :, :, None]/255.


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
batches = 8
datagen.fit(Xtrain)
num_batch = len(Xtrain)/batches
Progbar = tf.keras.utils.Progbar(num_batch)
for epoch in range(50):
    batch_count = 0
    model.save(r"C:\Users\User\Desktop\Super_AI\DL_NN\DL_VGG16\model")
    for x,y in datagen.flow(Xtrain,Ytrain,batch_size=batches):
        x = tf.image.resize(x,(224,224))
        x = tf.image.grayscale_to_rgb(x)
        history = model.fit(x,y,verbose=0)
        batch_count += 1
        Progbar.update(batch_count,values=[('loss',history.history['loss'][0])])
    if batch_count >= num_batch:
        print(epoch)
        break

Save model as .pd file

In [ ]:
model.save(r"C:\Users\User\Desktop\Super_AI\DL_NN\DL_A-Zmnist\Model")

loaded_model = tf.keras.models.load_model(r"C:\Users\User\Desktop\Super_AI\DL_NN\DL_A-Zmnist\Model")

##### Predict

In [ ]:
Z = loaded_model.predict(Xtest)
print('Performance ',np.sum(Z.argmax(axis=1)==Ytest)/len(Z))
# history = model
# plt.plot(history.history['loss'])
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.show()

In [ ]:
n = 10
print('Predict : ',Z[n])
print('Max : ',np.amax(Z[n]))
print('Predict index :',np.where(Z[n] == np.amax(Z[n]))[0],
        '| Predict Answer :',nr_to_letter.get(np.where(Z[n] == np.amax(Z[n]))[0][0]),
        '| Xtest :',Ytest[n])
plt.imshow(Xtest[n],cmap='gray')

#### Transfer Learning VGG16

In [21]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, Model

vgg = VGG16(include_top=False, weights='imagenet')
# fit input
x_in = layers.Input(shape=(32, 32, 1))
x = layers.Conv2D(3, 1)(x_in)
x = vgg(x)
# fit output
x = layers.Flatten()(x)
x = layers.Dense(10, activation='softmax')(x)
model = Model(x_in, x)
model.summary()

58900480/58889256 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 3)         6         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 14,719,824
Trainable params: 14,719,824
Non-trainable params: 0
_________________________________________________________________


In [22]:
Xtrain = tf.image.resize(Xtrain, (32, 32))

In [23]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')
model.fit(Xtrain, Ytrain, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 58s 30ms/step - loss: 0.1717
Epoch 2/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0298
Epoch 3/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0202
Epoch 4/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0151
Epoch 5/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0108
Epoch 6/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0092
Epoch 7/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0073
Epoch 8/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0063
Epoch 9/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0054
Epoch 10/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0043


In [ ]:
import numpy as np
Xtest = tf.image.resize(Xtest, (32, 32))
Ztest = model.predict(Xtest)
np.sum(Ztest.argmax(axis=1) == Ytest)/len(Ztest)
# 0.996

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

# fit output
x = layers.Flatten()(vgg.output)
x = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(vgg.input, x)
model.summary()

In [ ]:
Xtrain = tf.image.grayscale_to_rgb(Xtrain)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')
model.fit(Xtrain, Ytrain, epochs=10)

In [ ]:
Xtest = tf.image.grayscale_to_rgb(Xtest)
Ztest = model.predict(Xtest)
np.sum(Ztest.argmax(axis=1) == Ytest)/len(Ztest)
# 0.9951